In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
from tqdm import tqdm
import seaborn as sns
%matplotlib inline
import train_util
import util
%load_ext autoreload
%autoreload 2

from keras.models import load_model

/root/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
inter = np.histogram2d([0, 1, 0, 1], [1, 1, 0, 0], bins=(2, 2))

In [17]:
X_train, X_feat_train, Y_train, Y_no_salt, Y_salt_cls = train_util.get_data(data_type='train', size=(128, 128, 3))

100%|██████████| 3887/3887 [00:49<00:00, 78.74it/s]


In [21]:
inter2 = np.histogram2d(Y_train[0].flatten(), Y_train[0].flatten(), bins=(2, 2))

In [34]:
Y_train[:10].flatten().shape

(163840,)

In [91]:
a = np.array([[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0]])
b = np.array([[0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 1, 1]])

In [92]:
inter = np.histogram2d(a.flatten(), b.flatten(), bins=(2, 2))[0]

In [93]:
area_true = np.histogram(a, bins=2)[0]
area_pred = np.histogram(b, bins=2)[0]

In [94]:
area_true = np.expand_dims(area_true, -1)
area_pred = np.expand_dims(area_pred, 0)

In [95]:
area_true

array([[ 0],
       [24]])

In [105]:
inter = inter[1:, 1:]

In [97]:
area_pred

array([[10, 14]])

In [98]:
area_true + area_pred - inter

array([[10., 14.],
       [24., 24.]])

In [99]:
union = area_true + area_pred -inter

In [100]:
uni = union[1:, 1:]

In [101]:
uni[uni == 0]

array([], dtype=float64)

In [112]:
iou = inter / uni

In [116]:
matches = iou > 0.5

In [120]:
np.sum(matches, axis=0) == 0

array([False])

In [123]:
def iou_coef(y_true, y_pred, smooth=1):
    """
    IoU = (|X & Y|)/ (|X or Y|)
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum((y_true,-1) + K.sum(y_pred,-1)) - intersection
    return (intersection + smooth) / ( union + smooth)

In [124]:
# Immutable
変更不可　int float string tuple
mutable
list numpy array dict
# byval byref

In [163]:
def testFunc(x, y):
    print(id(x))
    print(id(y))
    x.append('hello')
    
    return c, d

In [164]:
a = [1, 2, 3, 4]

In [166]:
print(a)

[1, 2, 3, 4, 'hello']


In [165]:
f, g = testFunc(a, a)

140520806849672
140520806849672


NameError: name 'c' is not defined

In [131]:
# print(a, f, g)
# a [1, 2, 3, 4]
# f [1, 2, 3, 4, 'hello']
# g [1, 2, 3, 4, 'world']

In [141]:
print(a)

[1, 2, 3, 4, 'hello', 'world']


In [150]:
def test_func2(ab):
    ab += 3
    print(id(ab))
    
    return ab

In [151]:
g = 0

In [152]:
id(g)

140523687799296

In [153]:
cd = test_func2(g)

140523687799392


In [184]:
x_train = np.array([0, 1, 2, 3, 4, 5], dtype='float')

In [185]:
x_val = np.array([4, 1, 2, 8, 4, 6], dtype='float')

In [188]:
def norm(x_train, x_val):
    mean = x_train.mean()
    std = x_train.std()
    
    x_train -= mean
    x_train /= std
    
    x_val -= mean
    x_val /= std
    
    return x_train, x_val

In [189]:
test = norm(x_train, x_val)

In [193]:
x_test = norm(x_train, x_train)

In [194]:
x_train

array([-1.46385011, -0.87831007, -0.29277002,  0.29277002,  0.87831007,
        1.46385011])

In [143]:
 print(g)

0


In [195]:
import copy

In [197]:
copy_x = copy.deepcopy(x_train)

In [198]:
id(x_train)

140520807478576

In [199]:
id(copy_x)

140520807430144